In [1]:
import torch
import torch.nn as nn
from custom_dataloaders import construct_dataloaders
from hf_trainer import infer, train
from roberta_classification_model import RobertaClsModel
from torch import cuda
from transformers import AutoModelForSequenceClassification, AutoTokenizer

Choose your dataset. Make sure that the number of classes in your model matches the number of different labels in that dataset.

In [4]:
# AG News Dataset for classifying news headlines.
dataset_name = "ag_news"
dataset_num_labels = 4

# Uncomment the code below to use the SST2 dataset for sentiment analysis.
# NOTE: If you're going to use the SST2 dataset, you need to make sure that use_hf_sequence_classification = True
# The custom RoBERTa model is only defined for ag_news.
# NOTE: For SST2 to train well, you'll need to adjust the learning rate and weight decay in the hf_trainer file
# A good place to start is lr=0.00001, weight_decay=0.001
# dataset_name = "SetFit/sst2"
# dataset_num_labels = 2

Choose your pre-trained model and setup the dataloaders.

By default, the HuggingFace Transformer models will provide the dense hidden states of the last layer, one vector for each token in the input. These vectors are not directly usable for our task of classification at the sequence level. While they can be combined using the "attention mechanism" into a single class-specific sequence-level representation, we opt for an easier solution here.

This can be done by adding a "classification head"- a linear projection layer (`nn.Dense`)- on top of one of these token vectors in the output. For bi-directional encoder-only transformers such as BERT and RoBERTa, there is a specific token at the beginning of the input, \[CLS\], that representation the entire document. This layer will be added on top of the vector of the \[CLS\] token. For decoder-only transformers such as GPT and OPT, this projection layer might be added to the last non-pad token in the sentence.

The HuggingFace Transformers library provides a convenient way to add this layer to your pre-trained model. For a wide range of base models including RoBERTa and OPT, you can load the pre-trained model with the projection layer added and initialized for you using the `AutoModelForSequenceClassification` class:

```python
model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
```

To demonstrate how this useful abstraction works, we've manually added a classification head on top of a HuggingFace [**RoBERTa**](https://arxiv.org/abs/1907.11692) model in a custom torch.nn module. The RoBERTa model is very similar to the BERT model, with a few minor differences. For example the next-sentence prediction task was removed in pretraining of RoBERTa.

We encourage you to take a look at our implementation in *roberta_classification_model.py* and see whether the behavior differs from that of AutoModelForSequenceClassification. Note that there is also an implementation of the "decoder-only" style head in *gpt2_classification_model.py*.

Please note that if you need to experiment with a base model other than RoBERTa- for example, OPT- you will need to set `use_hf_sequence_classification = False` and use the HuggingFace AutoModelForSequenceClassification instead. 

In [5]:
# NOTE: If you're going to use the SST2 dataset, you need to make sure that use_hf_sequence_classification = True
# The custom RoBERTa model is only defined for ag_news
use_hf_sequence_classification = True  # set to True to use the HuggingFace abstraction
hf_model_name = "roberta-base"

# Uncomment the code below to use facebook/opt-125m as the base model.
# Note that using OPT-125m requires the use_hf_sequence_classification = True
# use_hf_sequence_classification = True
# hf_model_name = "facebook/opt-125m"  # Also try "facebook/opt-125m" for OPT.

In [7]:
# Create a tokenizer instance for a pretrained model vocabulary.
tokenizer = AutoTokenizer.from_pretrained(hf_model_name)

# Set the maximum number of tokens in each input.
tokenizer.model_max_length = 512
# Create data loader objects for train, validation, and test splits.
train_dataloader, val_dataloader, test_dataloader = construct_dataloaders(
    batch_size=8, train_split_ratio=0.8, tokenizer=tokenizer, dataset_name=dataset_name
)

Found cached dataset ag_news (/h/arashaf/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /h/arashaf/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-e3899bce123fc79a.arrow
Loading cached processed dataset at /h/arashaf/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-9deb23ce346562f6.arrow


Training data example encoding: tensor([    0, 22886,  1641,   354, 46707,     7,   492,  1920,   364,    12,
         6380,  9388,    13,   614,    12, 10111,  4247,    20,  6955,   935,
        22424,   189,   120,    10,   828,    55, 11138,   220,    76,   511,
            5,  2443,    14,  7299,  1641,   354, 46707,    18,   213, 11913,
         8569,   806,     6,  6431,    42,   186,     6,    40,   492,  1434,
            9,   614,    12, 10111,  1905,  4247,     5,   276,  1920,   364,
           12,  6380,  9388,    14,    16,   122,   129,   577,    15, 35456,
          906,  2793,  4247,     4,     2,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,    

Setup the different variables we'd like for training

In [8]:
device = "cuda" if cuda.is_available() else "cpu"
print(f"Detected Device {device}")
# We'll provide two options. First we create our own model on top of the vanilla RoBERTa model. The second is to use
# HuggingFace's AutoModel class, which essentially does the same thing for RoBERTa, but with support additional base
# models such as OPT and GPT-J.
classifier_model = (
    AutoModelForSequenceClassification.from_pretrained(hf_model_name, num_labels=dataset_num_labels)
    if use_hf_sequence_classification
    else RobertaClsModel()
)
loss_function = nn.CrossEntropyLoss()
n_training_epochs = 1
n_training_steps = 300

Detected Device cuda


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Train the model on the training dataset

In [9]:
print("Begin Model Training...")
# Initiates an Adam optimizer and runs the training loop.
train(
    classifier_model,
    train_dataloader,
    val_dataloader,
    loss_function,
    device,
    n_training_epochs,
    n_training_steps,
)
print("Training Complete")

Begin Model Training...
Starting Epoch 0
Completed batch number: 100 of 12000 in loader
Training Loss over last 100 steps: 1.075119908452034
Training Accuracy over last 100 steps: 57.301980198019805%
Validation Loss: 0.48847083688951004
Validation Accuracy: 83.08823529411765%
Completed batch number: 200 of 12000 in loader
Training Loss over last 100 steps: 0.42622626300901173
Training Accuracy over last 100 steps: 86.75%
Validation Loss: 0.29673660466191815
Validation Accuracy: 91.42156862745098%
Completed batch number: 300 of 12000 in loader
Training Loss over last 100 steps: 0.30904635064303876
Training Accuracy over last 100 steps: 90.0%
Validation Loss: 0.32352423938173874
Validation Accuracy: 89.2156862745098%
Training rounds complete. Validating on entire validation set.
Completed 300 of 3000...
Completed 600 of 3000...
Completed 900 of 3000...
Completed 1200 of 3000...
Completed 1500 of 3000...
Completed 1800 of 3000...
Completed 2100 of 3000...
Completed 2400 of 3000...
Complet

Once training is complete, we save the fine-tuned model to disk.

In [10]:
print("Saving model...")
hf_model_name_formatted = hf_model_name.split("/")[-1]
dataset_name_formatted = dataset_name.split("/")[-1]
output_model_file = f"./{hf_model_name_formatted}_{dataset_name_formatted}.bin"
torch.save(classifier_model, output_model_file)
print("Model saved to", output_model_file)

Saving model...
Model saved to ./roberta-base_ag_news.bin


Next, we load the model saved above, perform inference on the test set and measure loss and accuracy.

In [11]:
print("Loading model...")
classifier_model = torch.load(output_model_file)
print("Model loaded.")

print("Evaluating model on test set...")
test_accuracy, test_loss = infer(classifier_model, loss_function, test_dataloader, device)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}%")
print("Model evaluated.")

Loading model...
Model loaded.
Evaluating model on test set...
Completed 300 of 950...
Completed 600 of 950...
Completed 900 of 950...
Test Loss: 0.31733756162422266
Test Accuracy: 89.4342105263158%
Model evaluated.
